In [142]:
import numpy as np
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']


In [143]:
print('Is the mean of the training positions in y_train is zero?', np.isclose(np.mean(y_train), 0))
print('The mean of the positions in y_val are:', np.mean(y_val))
print('The standard error of the positions in y_val are:', np.std(y_val))
print('The mean of the first 5785 entries in the y_train is', np.mean(y_train[:5785]))
print('The standard error of the first 5785 entries in the y_train is', np.std(y_train[:5785]))

Is the mean of the training positions in y_train is zero? True
The mean of the positions in y_val are: -0.2160085093241599
The standard error of the positions in y_val are: 0.9814208579483531
The mean of the first 5785 entries in the y_train is -0.44247687859693674
The standard error of the first 5785 entries in the y_train is 0.907102593171408


In [144]:
# Detect and remove constant columns
constant_cols = np.where(np.all(X_train == X_train[0, :], axis=0))[0]

# Remove constant columns
X_train = np.delete(X_train, constant_cols, axis=1)
X_val = np.delete(X_val, constant_cols, axis=1)
X_test = np.delete(X_test, constant_cols, axis=1)


print("Removed constant columns:", constant_cols.tolist())

# Detect and remove duplicate columns
unique_cols, duplicate_cols = np.unique(X_train, axis=1, return_index=True)
duplicate_cols = np.setdiff1d(np.arange(X_train.shape[1]), duplicate_cols)

# Remove the columns
X_train = np.delete(X_train, duplicate_cols, axis=1)
X_val = np.delete(X_val, duplicate_cols, axis=1)
X_test = np.delete(X_test, duplicate_cols, axis=1)


print("Removed duplicate columns after first removing:", duplicate_cols.tolist())

Removed constant columns: [59, 69, 179, 189, 351]
Removed duplicate columns after first removing: [76, 77, 185, 195, 283, 354]


In [145]:
from ct_support_code import *

def fit_linreg(X, yy, alpha):
    # add a column of ones to the end of X
    X_fit = np.column_stack((X, np.ones(X.shape[0])))
    # add the regularization term to the X matrix
    alpha_matrix  = np.sqrt(alpha) * np.eye(X_fit.shape[1])
    # we ignore the bias b in the regularization
    alpha_matrix[-1, -1] = 0
    # add the regularization term to the X matrix
    X_hat = np.row_stack((X_fit, alpha_matrix))
    # add zeros to the end of y with number of coefficients
    y_hat = np.concatenate((y_train, np.zeros(X_fit.shape[1])))
    # solve Xw = y with w = [w, b]
    coeff = np.linalg.lstsq(X_hat, y_hat, rcond= None)[0] 
    b = coeff[-1]
    w = coeff[:-1]
    return w, b


# the rmse
def rmse(y, y_pred):
    return np.sqrt(np.mean((y - y_pred)**2))

# Linear regression with regularization
w, b = fit_linreg(X_train, y_train, 30)


y_train_pred_lg = np.dot(X_train, w) + b
y_val_pred_lg = np.dot(X_val, w) + b

# here we use support function fit_linreg_gradopt
wg, bg = fit_linreg_gradopt(X_train, y_train, 30)

y_train_pred_lgg = np.dot(X_train, wg) + bg
y_val_pred_lgg = np.dot(X_val, wg) + bg

print('The RMSE of fit_linreg on train data is:', rmse(y_train, y_train_pred_lg))
print('The RMSE of fit_linreg_gradopt on train data is:', rmse(y_train, y_train_pred_lgg))
print('The RMSE of fit_linreg on validation data is:', rmse(y_val, y_val_pred_lg))
print('The RMSE of fit_linreg_gradopt on validation data is:', rmse(y_val, y_val_pred_lgg))

The RMSE of fit_linreg on train data is: 0.3567565397204054
The RMSE of fit_linreg_gradopt on train data is: 0.3567556103401202
The RMSE of fit_linreg on validation data is: 0.4230521968394695
The RMSE of fit_linreg_gradopt on validation data is: 0.42305510586203865


In [146]:
def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)

# Store probabilities for each model
train_probs = np.zeros((X_train.shape[0], K))
val_probs = np.zeros((X_val.shape[0], K))
# record the ww and bb to V_3 and bk_3
V_3 = np.zeros(( K,X_train.shape[1]))
bk_3 = np.zeros(K)

for kk in range(K):
    labels = y_train > thresholds[kk]

    ww, bb = fit_logreg_gradopt(X_train, labels, 30)
    V_3[kk, :] = ww
    bk_3[kk] = bb

    # Preduct probabilities on the training and validation data
    train_probs[:, kk] = 1 / (1 + np.exp(-np.dot(X_train, ww) - bb))
    val_probs[:, kk] = 1 / (1 + np.exp(-np.dot(X_val, ww) - bb))

# Fit regularized linear regression on the transformed dataset
w_linear, b_linear = fit_linreg_gradopt(train_probs, y_train, alpha=30)

# record w_linear and b_linear to ww_3 and bb_3
ww_3 = w_linear
bb_3 = b_linear

# Calculate RMSE on the transformed training and validation sets
train_predictions = np.dot(train_probs, w_linear) + b_linear
val_predictions = np.dot(val_probs, w_linear) + b_linear

train_rmse = np.sqrt(np.mean((train_predictions - y_train) ** 2))
val_rmse = np.sqrt(np.mean((val_predictions - y_val) ** 2))

print("RMSE on transformed training data:", train_rmse)
print("RMSE on transformed validation data:", val_rmse)


RMSE on transformed training data: 0.15441185585977435
RMSE on transformed validation data: 0.25424947251332386


In [147]:
# fix the random seed
np.random.seed(0)
# set the hidden layers same to Q3
K = 20
D = X_train.shape[1]
# a)a sensible random initialization of the parameters
ww_init = np.random.randn(K)/np.sqrt(D+K)
bb_init = np.array(0)
V_init = np.random.randn(K, D)/np.sqrt(K+1)
bk_init = np.zeros(K)
init = (ww_init, bb_init, V_init, bk_init)

def fit_nn_gradopt(X, yy, alpha, K, D, init):
    args = (X, yy, alpha)
    ww, bb, V, bk = minimize_list(nn_cost, init, args)
    return ww, bb, V, bk

# Fit a neural network with 20 hidden units
ww, bb, V, bk = fit_nn_gradopt(X_train, y_train, 30, K, D, init)

# Predict on the training and validation data
y_train_pred_nn = nn_cost((ww, bb, V, bk), X_train)

# Calculate RMSE on the transformed training sets
train_rmse_nna = np.sqrt(np.mean((y_train_pred_nn - y_train) ** 2))
print("RMSE on transformed training data with random initialization:", train_rmse_nna)

# Predict on the validation data
y_val_pred_nn = nn_cost((ww, bb, V, bk), X_val)

# Calculate RMSE on the transformed validation sets
val_rmse_nna = np.sqrt(np.mean((y_val_pred_nn - y_val) ** 2))
print("RMSE on transformed validation data with random initialization:", val_rmse_nna)

# b) the parameters initialized using the fits made in Q3.
init = (ww_3, bb_3, V_3, bk_3)
ww, bb, V, bk = fit_nn_gradopt(X_train, y_train, 30, K, D, init)

# Predict on the training and validation data
y_train_pred_nn = nn_cost((ww, bb, V, bk), X_train)

# Calculate RMSE on the transformed training sets
train_rmse_nnb = np.sqrt(np.mean((y_train_pred_nn - y_train) ** 2))
print("RMSE on transformed training data with Q3 parameters:", train_rmse_nnb)

# Predict on the validation data
y_val_pred_nn = nn_cost((ww, bb, V, bk), X_val)

# Calculate RMSE on the transformed validation sets
val_rmse_nnb = np.sqrt(np.mean((y_val_pred_nn - y_val) ** 2))
print("RMSE on transformed validation data with Q3 parameters:", val_rmse_nnb)

# calculate the test data
y_test_pred_nn = nn_cost((ww, bb, V, bk), X_test)

# Calculate RMSE on the transformed test sets
test_rmse_nnb = np.sqrt(np.mean((y_test_pred_nn - y_test) ** 2))
print("RMSE on transformed test data with Q3 parameters:", test_rmse_nnb)

RMSE on transformed training data with random initialization: 0.1392111516735692
RMSE on transformed validation data with random initialization: 0.2718821406042969
RMSE on transformed training data with Q3 parameters: 0.13956257348153217
RMSE on transformed validation data with Q3 parameters: 0.2681698912659182
RMSE on transformed test data with Q3 parameters: 0.3014994363080638


In [148]:
# scipy.stats.norm.cdf
from scipy.stats import norm

def train_nn_reg(alpha):
    K = 20
    D = X_train.shape[1]
    
    ww_init = np.random.randn(K)/np.sqrt(D+K)
    bb_init = np.array(0)
    V_init = np.random.randn(K, D)/np.sqrt(K+1)
    bk_init = np.zeros(K)
    init = (ww_init, bb_init, V_init, bk_init)

    # Fit a neural network with 20 hidden units
    ww, bb, V, bk = fit_nn_gradopt(X_train, y_train, alpha, K, D, init)

    # Predict on the validation data
    y_val_pred_nn = nn_cost((ww, bb, V, bk), X_val)

    # Calculate RMSE on the transformed validation sets
    val_rmse_nn = np.sqrt(np.mean((y_val_pred_nn - y_val) ** 2))
    return val_rmse_nn

# alpha from 0 to 50 with step 0.02
alphas = np.arange(0, 50, 0.02)

# pick three alpha values to treat as training locations
train_alphas = np.array([6, 25, 40])
train_index = np.array([300, 1250, 2000])

# use the remaining locations as values that you will consider with the acquisition function
# remove the train_alphas from alphas
alpha_acq = np.delete(alphas, train_index)

# calculate y1, y2 and y3
y1 = np.log(val_rmse_nna) - np.log(train_nn_reg(train_alphas[0]))
y2 = np.log(val_rmse_nna) - np.log(train_nn_reg(train_alphas[1]))
y3 = np.log(val_rmse_nna) - np.log(train_nn_reg(train_alphas[2])) 

yy = np.array([y1, y2, y3])

def PI(mu, sd, y_obs):
    return norm.cdf((mu - np.max(y_obs)) / sd)

# iterate 5 times
for i in range(1,6):

    # find the posterior mean and variance
    rest_cond_mu, rest_cond_cov = gp_post_par(alpha_acq, train_alphas, yy)

    # find the standard deviation of each point
    sd = np.sqrt(np.diag(rest_cond_cov))

    # find the maximum value of PI
    for j in range(len(alpha_acq)):
        PI_val = PI(rest_cond_mu[j], sd[j], yy)
        if j == 0:
            max_PI = PI_val
            max_alpha = alpha_acq[j]
            max_index = j
        elif PI_val > max_PI:
            max_PI = PI_val
            max_alpha = alpha_acq[j]
            max_index = j
    
    print('The maximum PI value in ', str(i), 'iteration :', max_PI)
    print('The maximum alpha in ', str(i), 'iteration :', max_alpha)

    # add the max_alpha to train_alphas
    train_alphas = np.append(train_alphas, max_alpha)
    # remove the max_alpha from alpha_acq
    alpha_acq = np.delete(alpha_acq, max_index)

    # calculate the new y value
    y_new = np.log(val_rmse_nna) - np.log(train_nn_reg(max_alpha))
    yy = np.append(yy, y_new)

# find the best alpha
best_alpha = train_alphas[np.argmax(yy)]
print('The best alpha is:', best_alpha)

# find its validation RMSE
best_val_rmse = train_nn_reg(best_alpha)
print('The validation RMSE of the best alpha is:', best_val_rmse) 

# find the test RMSE
K = 20
D = X_train.shape[1]
ww_init = np.random.randn(K)/np.sqrt(D+K)
bb_init = np.array(0)
V_init = np.random.randn(K, D)/np.sqrt(K+1)
bk_init = np.zeros(K)
init = (ww_init, bb_init, V_init, bk_init)


# Fit a neural network with 20 hidden units
ww, bb, V, bk = fit_nn_gradopt(X_train, y_train, best_alpha, K, D, init)

# Predict on the test data
y_test_pred_nn = nn_cost((ww, bb, V, bk), X_test)

# Calculate RMSE on the test sets
test_rmse = np.sqrt(np.mean((y_test_pred_nn - y_test) ** 2))
print("RMSE on test data with the best alpha:", test_rmse)


The maximum PI value in  1 iteration : 0.27484055761808257
The maximum alpha in  1 iteration : 6.22
The maximum PI value in  2 iteration : 0.18140627820261634
The maximum alpha in  2 iteration : 2.2800000000000002
The maximum PI value in  3 iteration : 0.13519171829121923
The maximum alpha in  3 iteration : 10.4
The maximum PI value in  4 iteration : 0.11539743542363978
The maximum alpha in  4 iteration : 2.82
The maximum PI value in  5 iteration : 0.16773646902592754
The maximum alpha in  5 iteration : 2.84
The best alpha is: 6.0
The validation RMSE of the best alpha is: 0.24658887961477222
RMSE on test data with the best alpha: 0.29067703512841714


In [151]:

def nn_cost_new(params, X, yy=None, alpha=None):
    # Unpack parameters from list
    ww, bb, V, bk, M, bm, Q, bq = params

    # Forwards computation of cost 
    L0 = np.dot(X, Q.T) + bq[None,:] # N,P
    P0 = np.maximum(0, L0) # N,P
    L1 = np.dot(P0, M.T) + bm[None,:] # N,M
    P1 = np.maximum(0, L1) # N,M 
    L2 = np.dot(P1, V.T) + bk[None,:] # N,K
    P2 = 1 / (1 + np.exp(-L2)) # N,K
    F = np.dot(P2, ww) + bb # N,
    if yy is None:
        # user wants prediction rather than training signal:
        return F
    res = F - yy # N,
    E = np.dot(res, res) + alpha*(np.sum(Q*Q)+ np.sum(M*M) + np.sum(V*V) + np.dot(ww,ww)) # 1x1

    # Reverse computation of gradients
    F_bar = 2*res # N,
    ww_bar = np.dot(P2.T, F_bar) + 2*alpha*ww # K,
    bb_bar = np.sum(F_bar) # scalar
    P2_bar = np.dot(F_bar[:,None], ww[None,:]) # N,K
    L2_bar = P2_bar * P2 * (1 - P2) # N,K
    V_bar = np.dot(L2_bar.T, P1) + 2*alpha*V # K,D
    bk_bar = np.sum(L2_bar, 0)
    P1_bar = np.dot(L2_bar, V) # N,M
    L1_bar = P1_bar * (L1 > 0) # N,M
    M_bar = np.dot(L1_bar.T, P0) + 2*alpha*M # M,P
    bm_bar = np.sum(L1_bar, 0)
    P0_bar = np.dot(L1_bar, M) # N,P
    L0_bar = P0_bar * (L0 > 0) # N,P
    Q_bar = np.dot(L0_bar.T, X) + 2*alpha*Q # P,D


    return E, (ww_bar, bb_bar, V_bar, bk_bar, M_bar, bm_bar, Q_bar, bq)

# set the hidden layers same to Q3
K = [128,64, 32]
D = X_train.shape[1]
# a)a sensible random initialization of the parameters
ww_init = np.random.randn(K[2])/np.sqrt(1+K[2])
#bb_init = np.random.randn()/np.sqrt(D+K)
bb_init = np.array(0)

V_init = np.random.randn(K[2], K[1])/np.sqrt(K[2]+K[1])
bk_init = np.zeros(K[2])
#bk_init = np.random.randn(K)/np.sqrt(K)
M_init = np.random.randn(K[1], K[0])/np.sqrt(K[0]/2)
bm_init = np.zeros(K[1])
#bm_init = np.random.randn(D)/np.sqrt(D+1)
Q_init = np.random.randn(K[0], D)/np.sqrt(D/2)
bq_init = np.zeros(K[0])

init = (ww_init, bb_init, V_init, bk_init, M_init, bm_init, Q_init, bq_init)

def fit_nn_gradopt_new(X, yy, alpha, K, D, init):
    args = (X, yy, alpha)
    ww, bb, V, bk, M, bm, Q, bq = minimize_list(nn_cost_new, init, args)
    return ww, bb, V, bk, M, bm, Q, bq

# fit the model
ww, bb, V, bk, M, bm, Q, bq = fit_nn_gradopt_new(X_train, y_train, 6.4, K, D, init)

# Predict on the training set
y_train_pred_nn = nn_cost_new((ww, bb, V, bk, M, bm, Q, bq), X_train)

# Calculate RMSE on the transformed training sets
train_rmse_nna = np.sqrt(np.mean((y_train_pred_nn - y_train) ** 2))
print("RMSE on transformed training data with random initialization:", train_rmse_nna)


# Predict on the validation set
y_val_pred_nn = nn_cost_new((ww, bb, V, bk, M, bm, Q, bq), X_val)

# Calculate RMSE on the transformed validation sets
val_rmse_nna = np.sqrt(np.mean((y_val_pred_nn - y_val) ** 2))
print("RMSE on transformed validation data with random initialization:", val_rmse_nna)

# test on the test set
y_test_pred_nn = nn_cost_new((ww, bb, V, bk, M, bm, Q, bq), X_test)

# Calculate RMSE on the transformed test sets
test_rmse_nna = np.sqrt(np.mean((y_test_pred_nn - y_test) ** 2))
print("RMSE on transformed test data with random initialization:", test_rmse_nna)

RMSE on transformed training data with random initialization: 0.025587545888610508
RMSE on transformed validation data with random initialization: 0.21954907757778808
RMSE on transformed test data with random initialization: 0.2349335232342645
